In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [2]:
df = pd.read_csv("Dataset/dataset_fix.csv")
df

,Kalimat #,Word,Tag
0,0,tiap,O
1,0,makan,O
2,0,kesini,O
3,0,ngga,O
4,0,pernah,O
...,...,...,...
50531,859,portions,I-FOOD
50532,859,way,I-FOOD
50533,859,too,I-FOOD
50534,859,small,I-FOOD


In [3]:
dic={}
for i, tag in enumerate(df.Tag.unique()):
    dic[tag] = i
dic

{'O': 0,
 'B-FOOD': 1,
 'I-FOOD': 2,
 'B-MISCELLANEOUS': 3,
 'I-MISCELLANEOUS': 4,
 'B-SERVICE': 5,
 'I-SERVICE': 6,
 'B-AMBIENCE': 7,
 'I-AMBIENCE': 8,
 'B-PRICE': 9,
 'I-PRICE': 10}

In [4]:
labels = df["Tag"].apply(lambda x:dic[x])
labels

0        0
1        0
2        0
3        0
4        0
        ..
50531    2
50532    2
50533    2
50534    2
50535    0
Name: Tag, Length: 50536, dtype: int64

In [6]:
list_kalimat = []
for i in range(df["Kalimat #"].min(),df["Kalimat #"].max()+1):
#     list_kata = ["<S>"]
    list_kata = []
    for kata in df[df["Kalimat #"] == i]["Word"]:
        list_kata.append(str(kata))
#     list_kata.append("</S>")
    list_kalimat.append(list_kata)

In [7]:
list_kalimat_join = []
for kalimat in list_kalimat:
    list_kalimat_join.append(" ".join(kalimat))

In [8]:
print(list_kalimat[0])

['tiap', 'makan', 'kesini', 'ngga', 'pernah', 'cuma', '1', 'atau', '2', 'porsi', 'pasti', 'nambah', 'terus', 'karena', 'emang', 'pas', 'banget', 'rasanya', 'di', 'lidah', 'black', 'peppernya', 'yang', 'paling', 'enak', 'disini', 'emang', 'selalu', 'waiting', 'list', 'cuma', 'sei', 'sapi', 'lamalera', 'absolutely', 'worth', 'to', 'wait', 'recommended']


In [9]:
list_kalimat_join[0]

'tiap makan kesini ngga pernah cuma 1 atau 2 porsi pasti nambah terus karena emang pas banget rasanya di lidah black peppernya yang paling enak disini emang selalu waiting list cuma sei sapi lamalera absolutely worth to wait recommended'

In [10]:
NUM_WORDS=10000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-.:;=?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(list_kalimat)
sequences_train = tokenizer.texts_to_sequences(list_kalimat)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 7552 unique tokens.


In [12]:
word_index

{'dan': 1,
 'yang': 2,
 'the': 3,
 'enak': 4,
 'di': 5,
 'nya': 6,
 'juga': 7,
 'banget': 8,
 'and': 9,
 'ini': 10,
 'yg': 11,
 'ada': 12,
 'untuk': 13,
 'tapi': 14,
 'ga': 15,
 'saya': 16,
 'i': 17,
 'to': 18,
 'bandung': 19,
 'is': 20,
 'dengan': 21,
 'a': 22,
 'makan': 23,
 'buat': 24,
 'sih': 25,
 'tempatnya': 26,
 'ke': 27,
 'karena': 28,
 'rasa': 29,
 'sama': 30,
 'tempat': 31,
 'was': 32,
 'dari': 33,
 'pas': 34,
 'banyak': 35,
 'of': 36,
 'rasanya': 37,
 'tidak': 38,
 'it': 39,
 'jadi': 40,
 'aku': 41,
 'menu': 42,
 'for': 43,
 'disini': 44,
 'bgt': 45,
 'lagi': 46,
 'makanan': 47,
 'agak': 48,
 'kesini': 49,
 'bisa': 50,
 'kalo': 51,
 'aja': 52,
 'good': 53,
 'suka': 54,
 'place': 55,
 'in': 56,
 'with': 57,
 'but': 58,
 'pesen': 59,
 'so': 60,
 'harga': 61,
 'lebih': 62,
 'ya': 63,
 'itu': 64,
 'cukup': 65,
 'terlalu': 66,
 'kalau': 67,
 'lumayan': 68,
 'food': 69,
 'not': 70,
 'kurang': 71,
 'sangat': 72,
 'you': 73,
 'mau': 74,
 'paling': 75,
 'this': 76,
 'nyaman': 77,
 'n

In [13]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
toEncode = labels.values.reshape(-1, 1)
enc = enc.fit(toEncode)
enc.get_feature_names()

array(['x0_0', 'x0_1', 'x0_2', 'x0_3', 'x0_4', 'x0_5', 'x0_6', 'x0_7',
       'x0_8', 'x0_9', 'x0_10'], dtype=object)

In [14]:
Encoded = enc.transform(toEncode).toarray()
Encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [20]:
token_encoded = []
for i in sequences_train:
    for j in i:
        token_encoded.append([j])

In [22]:
len(token_encoded)

50536

In [23]:
len(Encoded)

50536

In [24]:
X_train, X_test, y_train, y_test = train_test_split(token_encoded, Encoded, test_size=0.2, random_state=1301170066, shuffle=True, stratify=labels.values)

In [25]:
X_train = np.array([np.array(x).astype('float32') for x in X_train])
y_train = np.array([np.array(x).astype('float32') for x in y_train])

X_test = np.array([np.array(x).astype('float32') for x in X_test])
y_test = np.array([np.array(x).astype('float32') for x in y_test])

In [26]:
print('Shape of X train:', X_train.shape)
print('Shape of label train:', y_train.shape)

print('Shape of X test:', X_test.shape)
print('Shape of label test:', y_test.shape)

Shape of X train: (40428, 1)
Shape of label train: (40428, 11)
Shape of X test: (10108, 1)
Shape of label test: (10108, 11)


In [27]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

idwiki_300 = Word2Vec.load("Model/idwiki_word2vec_300.model")
word_vectors = idwiki_300.wv

EMBEDDING_DIM = 300
vocabulary_size = min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i] = np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [28]:
X_train.shape

(40428, 1)

In [32]:
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout, concatenate, LSTM
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]
filter_sizes = [1,2,3]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=11, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)

In [33]:
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 300)       2265900     input_3[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 1, 300, 1)    0           embedding[2][0]                  
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 1, 1, 100)    30100       reshape_2[0][0]                  
______________________________________________________________________________________________

In [38]:
model.fit(X_train, y_train, 
          batch_size=1000, 
          epochs=100, 
          verbose=1, 
          validation_data=(X_test, y_test),
          # callbacks=callbacks
         )

Epoch 1/100
41/41 [==============================] - 3s 68ms/step - loss: 0.9854 - acc: 0.7193 - val_loss: 1.0536 - val_acc: 0.7160
Epoch 2/100
41/41 [==============================] - 3s 72ms/step - loss: 0.9754 - acc: 0.7235 - val_loss: 1.0516 - val_acc: 0.7157
Epoch 3/100
41/41 [==============================] - 3s 62ms/step - loss: 0.9701 - acc: 0.7243 - val_loss: 1.0488 - val_acc: 0.7189
Epoch 4/100
41/41 [==============================] - 3s 70ms/step - loss: 0.9639 - acc: 0.7257 - val_loss: 1.0470 - val_acc: 0.7170
Epoch 5/100
41/41 [==============================] - 3s 66ms/step - loss: 0.9594 - acc: 0.7265 - val_loss: 1.0449 - val_acc: 0.7188
Epoch 6/100
41/41 [==============================] - 3s 67ms/step - loss: 0.9524 - acc: 0.7302 - val_loss: 1.0454 - val_acc: 0.7189
Epoch 7/100
41/41 [==============================] - 3s 68ms/step - loss: 0.9500 - acc: 0.7276 - val_loss: 1.0455 - val_acc: 0.7194
Epoch 8/100
41/41 [==============================] - 3s 70ms/step - loss: 0.

41/41 [==============================] - 3s 64ms/step - loss: 0.8474 - acc: 0.7504 - val_loss: 1.0186 - val_acc: 0.7253
Epoch 63/100
41/41 [==============================] - 3s 67ms/step - loss: 0.8470 - acc: 0.7499 - val_loss: 1.0160 - val_acc: 0.7260
Epoch 64/100
41/41 [==============================] - 3s 69ms/step - loss: 0.8461 - acc: 0.7504 - val_loss: 1.0159 - val_acc: 0.7267
Epoch 65/100
41/41 [==============================] - 3s 68ms/step - loss: 0.8476 - acc: 0.7502 - val_loss: 1.0182 - val_acc: 0.7228
Epoch 66/100
41/41 [==============================] - 3s 67ms/step - loss: 0.8437 - acc: 0.7506 - val_loss: 1.0184 - val_acc: 0.7185
Epoch 67/100
41/41 [==============================] - 3s 61ms/step - loss: 0.8439 - acc: 0.7495 - val_loss: 1.0141 - val_acc: 0.7229
Epoch 68/100
41/41 [==============================] - 3s 67ms/step - loss: 0.8424 - acc: 0.7495 - val_loss: 1.0131 - val_acc: 0.7245
Epoch 69/100
41/41 [==============================] - 3s 64ms/step - loss: 0.8435 

In [39]:
y_pred = model.predict(X_test)

In [40]:
y_test_argmax = [np.argmax(i) for i in y_test]
y_pred_argmax = [np.argmax(i) for i in y_pred]

In [41]:
from sklearn.metrics import classification_report, make_scorer

print(classification_report(
    y_test_argmax, y_pred_argmax, digits=3))

              precision    recall  f1-score   support

           0      0.757     0.945     0.841      7014
           1      0.415     0.172     0.243       395
           2      0.494     0.282     0.359      1360
           3      0.000     0.000     0.000       110
           4      0.350     0.103     0.159       399
           5      0.568     0.292     0.385        72
           6      0.580     0.153     0.242       261
           7      0.400     0.026     0.048        78
           8      0.444     0.193     0.269       249
           9      0.824     0.326     0.467        43
          10      0.642     0.268     0.378       127

    accuracy                          0.721     10108
   macro avg      0.498     0.251     0.308     10108
weighted avg      0.666     0.721     0.670     10108



In [43]:
n = 5555
print("Kata:", X_test[n])
print("Label:", np.argmax(y_test[n]))
print("Prediksi:", np.argmax(y_pred[n]))

Kata: [58.]
Label: 0
Prediksi: 0
